This notebook illustrates how to train a NER model using the well known CONLL dataset, and sklearn_crfsuite library. 

### Importing Necessary Libraries

In [10]:
# Necessary imports
import nltk
from nltk.tag import pos_tag
from sklearn_crfsuite import CRF, metrics
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Download the NLTK tagger resource
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/scullyz/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Loading The Data

In [11]:
"""
Load the training/testing data. 
input: conll format data, but with only 2 tab separated columns - words and NE tags.
output: A list where each item is 2 lists. sentence as a list of tokens, NER tags as a list for each token.
"""
def load__data_conll(file_path):
    myoutput, words, tags = [], [], []
    with open(file_path, 'r') as fh:
        for line in fh:
            line = line.strip()
            if "\t" not in line:
                # Sentence ended.
                if words:
                    myoutput.append([words, tags])
                words, tags = [], []
            else:
                word, tag = line.split("\t")
                words.append(word)
                tags.append(tag)
    return myoutput

In [12]:
"""
Get features for all words in the sentence
Features:
- word context: a window of 2 words on either side of the current word, and current word.
- POS context: a window of 2 POS tags on either side of the current word, and current tag. 
input: sentence as a list of tokens.
output: list of dictionaries. each dict represents features for that word.
"""
def sent2feats(sentence):
    feats = []
    sen_tags = pos_tag(sentence)  # POS tagging the sentence
    for i in range(len(sentence)):
        word = sentence[i]
        wordfeats = {}
        # word features: word, prev 2 words, next 2 words in the sentence.
        wordfeats['word'] = word
        if i == 0:
            wordfeats["prevWord"] = wordfeats["prevSecondWord"] = "<S>"
        elif i == 1:
            wordfeats["prevWord"] = sentence[0]
            wordfeats["prevSecondWord"] = "</S>"
        else:
            wordfeats["prevWord"] = sentence[i - 1]
            wordfeats["prevSecondWord"] = sentence[i - 2]

        if i == len(sentence) - 2:
            wordfeats["nextWord"] = sentence[i + 1]
            wordfeats["nextNextWord"] = "</S>"
        elif i == len(sentence) - 1:
            wordfeats["nextWord"] = "</S>"
            wordfeats["nextNextWord"] = "</S>"
        else:
            wordfeats["nextWord"] = sentence[i + 1]
            wordfeats["nextNextWord"] = sentence[i + 2]

        # POS tag features: current tag, previous and next 2 tags.
        wordfeats['tag'] = sen_tags[i][1]
        if i == 0:
            wordfeats["prevTag"] = wordfeats["prevSecondTag"] = "<S>"
        elif i == 1:
            wordfeats["prevTag"] = sen_tags[0][1]
            wordfeats["prevSecondTag"] = "</S>"
        else:
            wordfeats["prevTag"] = sen_tags[i - 1][1]
            wordfeats["prevSecondTag"] = sen_tags[i - 2][1]

        if i == len(sentence) - 2:
            wordfeats["nextTag"] = sen_tags[i + 1][1]
            wordfeats["nextNextTag"] = "</S>"
        elif i == len(sentence) - 1:
            wordfeats["nextTag"] = "</S>"
            wordfeats["nextNextTag"] = "</S>"
        else:
            wordfeats["nextTag"] = sen_tags[i + 1][1]
            wordfeats["nextNextTag"] = sen_tags[i + 2][1]

        feats.append(wordfeats)
    return feats

### Extracting Features

In [13]:
# Extract features from the conll data, after loading it.
def get_feats_conll(conll_data):
    feats = []
    labels = []
    for sentence in conll_data:
        feats.append(sent2feats(sentence[0]))
        labels.append(sentence[1])
    return feats, labels

### Confusion matrix printer

In [14]:
def print_cm(cm, labels):
    columnwidth = max([len(x) for x in labels] + [5])
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=" ")
    for label in labels:
        print(f"{label:{columnwidth}}", end=" ")
    print()
    
    for i, label1 in enumerate(labels):
        print(f"{label1:{columnwidth}}", end=" ")
        for j in range(len(labels)):
            cell = f"{cm[i, j]:{columnwidth}.0f}"
            print(cell, end=" ")
        print()

### Generate confusion matrix

In [16]:
def get_confusion_matrix(y_true, y_pred, labels):
    trues, preds = [], []
    for yseq_true, yseq_pred in zip(y_true, y_pred):
        trues.extend(yseq_true)
        preds.extend(yseq_pred)
    
    # Fix: Explicitly pass the 'labels' as a keyword argument
    cm = confusion_matrix(trues, preds, labels=labels)
    print_cm(cm, labels)

### Training a Model

In [17]:
# Train a sequence model
def train_seq(X_train, Y_train, X_dev, Y_dev):
    crf = CRF(algorithm='lbfgs', c1=0.1, c2=10, max_iterations=50)
    crf.fit(X_train, Y_train)
    labels = list(crf.classes_)
    y_pred = crf.predict(X_dev)
    sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))

    print(metrics.flat_f1_score(Y_dev, y_pred, average='weighted', labels=labels))
    print(metrics.flat_classification_report(Y_dev, y_pred, labels=sorted_labels, digits=3))
    
    # Fix: Correct call to get_confusion_matrix
    get_confusion_matrix(Y_dev, y_pred, labels=sorted_labels)

### Call all our functions inside the main method

In [18]:
# Main function
def main():
    try:
        from google.colab import files
        uploaded = files.upload()
        train_path = 'train.txt'
        test_path = 'test.txt'
    except:
        train_path = '/Users/scullyz/Desktop/NER/conlldata/train.txt'
        test_path = '/Users/scullyz/Desktop/NER/conlldata/test.txt'
    
    conll_train = load__data_conll(train_path)
    conll_dev = load__data_conll(test_path)
    
    print("Training a Sequence classification model with CRF")
    feats, labels = get_feats_conll(conll_train)
    devfeats, devlabels = get_feats_conll(conll_dev)
    train_seq(feats, labels, devfeats, devlabels)
    print("Done with sequence model")

if __name__ == "__main__":
    main()

Training a Sequence classification model with CRF
0.9255103670420659
              precision    recall  f1-score   support

           O      0.973     0.981     0.977     38323
       B-LOC      0.694     0.765     0.728      1668
       I-LOC      0.738     0.482     0.584       257
      B-MISC      0.648     0.309     0.419       702
      I-MISC      0.626     0.505     0.559       216
       B-ORG      0.670     0.561     0.611      1661
       I-ORG      0.551     0.704     0.618       835
       B-PER      0.773     0.766     0.769      1617
       I-PER      0.819     0.886     0.851      1156

    accuracy                          0.928     46435
   macro avg      0.721     0.662     0.679     46435
weighted avg      0.926     0.928     0.926     46435

           O      B-LOC  I-LOC  B-MISC I-MISC B-ORG  I-ORG  B-PER  I-PER  
O       37579    118      3     22     32    193    224     88     64 
B-LOC     143   1276      1     36      1     95     14     98      4 
I-LOC    

This is pretty good. We already have a model which has an F-score of 92%!!!